In [1]:
import random, gc, os, pickle, csv
import datasets.utils
from torch.utils import data

import pandas as pd
import numpy as np
# pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None)

In [2]:
dataset_order_mapping = {
    1: [2, 0, 3, 1, 4],
    2: [3, 4, 0, 1, 2],
    3: [2, 4, 1, 3, 0],
    4: [0, 2, 1, 4, 3]
}
use_db_cache = True
cache_dir = 'tmp'

In [3]:
args = {
    "order": 1,
    "n_epochs": 1,
    "lr": 3e-5,
    "inner_lr": 0.001*50,
    "meta_lr": 3e-5,
    "model": "bert",
    "learner": "oml",
    "mini_batch_size": 16,
    "updates": 5*2,
    "write_prob": 1.0,
    "max_length": 448,
    "seed": 42,
    "replay_rate": 0.01,
    "replay_every": 9600
}
updates = args["updates"]
mini_batch_size = args["mini_batch_size"]
order = args["order"]

# Make Data
Run this only when you need to make new data on `tmp/all_text_data.csv`

In [4]:
# print('Loading the datasets')
# test_datasets = []
# for dataset_id in dataset_order_mapping[order]:
#     test_dataset_file = os.path.join(cache_dir, f"{dataset_id}.cache")
#     if os.path.exists(test_dataset_file):
#         with open(test_dataset_file, 'rb') as f:
#             test_dataset = pickle.load(f)
#     else:
#         test_dataset = datasets.utils.get_dataset_test("", dataset_id)
#         print('Loaded {}'.format(test_dataset.__class__.__name__))
#         test_dataset = datasets.utils.offset_labels(test_dataset)
#         pickle.dump(test_dataset, open( test_dataset_file, "wb" ), protocol=pickle.HIGHEST_PROTOCOL)
#         print(f"Pickle saved at {test_dataset_file}")
#     test_datasets.append(test_dataset)
# print('Finished loading all the datasets')


# data_for_export = []
# for test_dataset in test_datasets:
#     print('Testing on {}'.format(test_dataset.__class__.__name__))
#     test_dataloader = data.DataLoader(test_dataset, batch_size=mini_batch_size, shuffle=False,
#                                       collate_fn=datasets.utils.batch_encode)
#     all_text, all_labels = [],[]
#     for text, labels in test_dataloader:
#         all_text.extend(text)
#         all_labels.extend(labels)
        
#     data_ids = [test_dataset.__class__.__name__ + str(i) for i in range(len(all_labels))]
#     data_for_export.extend(list(zip(data_ids, all_text, all_labels)))

# csv_filename = os.path.join(cache_dir, "all_text_data.csv")
# with open(csv_filename, 'w') as csv_file:
#     csv_writer = csv.writer(csv_file)
#     csv_writer.writerow(["data_idx", "text", "label"])
#     csv_writer.writerows(data_for_export)
# print(f"Done writing CSV File at {csv_filename}")

In [5]:
all_data_df = pd.read_csv("tmp/all_text_data.csv")
all_data_df = all_data_df[["data_idx", "text"]]
# all_data_df

# Analyze Data

In [33]:
n_adapt_csv = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241546_update0_results.csv"
adapt_csv = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241546_update5_results.csv" # Adapt update 5
# adapt_csv = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241546_update10_results.csv" # Adapt update 10

label_id = 30

n_adapt_df = pd.read_csv(n_adapt_csv)
adapt_df = pd.read_csv(adapt_csv)

df1 = n_adapt_df[n_adapt_df["label"]==label_id]
df2 = adapt_df[adapt_df["label"]==label_id][["label_conf", "pred"]].rename(columns={"label_conf": "label_conf2", "pred": "pred2"})
# all_data_df = all_data_df[all_data_df["label"]==label_id]["text"]
combined_df = pd.concat([df1, df2], axis=1)
combined_df = pd.merge(combined_df, all_data_df, how='inner', on = 'data_idx') # Can merge or just concat on top

# combined_df
npa = (combined_df["label_conf"] + combined_df["label_conf2"]).rename("npa") # n+a
nma = (combined_df["label_conf"] - combined_df["label_conf2"]).rename("nma") # n-a
combined_df = pd.concat([combined_df, npa, nma], axis=1)

# combined_df.sort_values(by=['npa'], ascending=True)[:10] # min n+a (all NOT conf)
# combined_df.sort_values(by=['npa'], ascending=False)[:10] # max n+a (all conf)
combined_df.sort_values(by=['nma'], ascending=True)[:10] # min n-a  (good Adapt)
# combined_df.sort_values(by=['nma'], ascending=False)[:10] # max n-a  (worse adapt)

,data_idx,label,label_conf,pred,label_conf2,pred2,text,npa,nma
126,YahooAnswersDataset1229,30,0.007635,27,0.007479,27,"does anyone know a site i can use to have free downloads of music?i need something to download free music on!peer to peer networks before you do let me offer alternatives that are safer. to ignore scroll down! be sure to checkout first: / / / / / / winamp has free shoutcast tv and audio stations / check out this page on how to watch it in winamp checkout this site to watch tv online / for free music videos and audio shorts if worried about legal stuff, for paid cheap music try: / / / / --- ---- --- --- end alternatives before thinking about downloading be sure to read the legal implications of downloading in the wikipedia p2p article be sure to have a descent spyware and virus program or you can scan online free in multiple sources. make sure you know about safety because many peer to peer networks have viruses. the link below has many resources including free antiviruses. before you start downloading you should know how to configure your router. to do so check here / currently there are two main p2p sources bittorrent and file sharing programs. to use bit torrent you need to download a client like azureus or bitcomet / / bit torrent in order of popularity: / / / / / / programs in order of popularity / emule is the most downloaded program in sourceforge history limewire is popular among kids / frostwire is an alternative to limewire / free filesharing / another gnutella client / phex is an alternative to bearshare and uses the gnutella network regarded as ""safer"". shareaza imesh / direct connect++ strong direct connect morpheus / warez p2p / sourceforge top filesharing projects wikipedia p2p list sourceforge free software list (often p2p resources) free music from archive.org programs to make sure all video/audio files work try these: media player classic is fast, free and much better than windows media player mplayer vcl / xp codec pack / real alternative divx free xvid quicktime alterna",0.015115,0.000156
438,YahooAnswersDataset4296,30,0.001824,26,0.001602,26,how long does it take to get a mba?please answers only1 year if you get into the 1 year program 2 if you get into the 2 year program 3 depending on you getting into the 3 year program lot depends on what you did before you applied for the mba. like if you did a bba or if you started a company like microsoft you might even get phd in business administration without having attended a single class. work experience adds value in getting the right program. but if you are starting from your kindergarten it will probably take you about 15 years to get there.,0.003425,0.000222
348,YahooAnswersDataset3458,30,0.007340,27,0.007078,27,"msn messenger help?msn is showing my contacts that i have a webcam, but i dont have a webcam! help!mine dose that, its easy to get 'rid of' click on 'personal settings' then on the personal tab, look down to the bottem of that box then you should see 'allow others too see that i have a webcam' then un-check it, there there you are :d hope this helps :d",0.014418,0.000262
440,YahooAnswersDataset4326,30,0.009453,27,0.009121,27,"what type of codec do i need to download to watch and burn movies from safe-share?trying to learn about codectry this site, they include a movie viewer with a bundle of codecs.all you will need in one.",0.018574,0.000332
660,YahooAnswersDataset6318,30,0.012192,27,0.011825,27,"where can i download a movie from? i need something free and safe from adware and spyware?i would like to download some new and good movies,for free.do you know any site? thanks.for spyware solutions, spybot search & destroy works very well. if you are willing to try microsoft's beta, you can also consider defender. both are currently free. always remember that an ounce of prevention is worth more than a pound of cure. that said, do not visit unknown websites, open suspicious emails and attachments, or click on links that you don’t know w